## Xây dựng Search Engine trên sàn thương mại điện tử Amazon
### 1. Sử dụng kỹ thuật Web Scraping để lấy dữ liệu sản phẩm từ sàn amazon

1. Thêm thư viện [Selenium](https://www.selenium.dev/) và các thư viện cần thiết khác.
2. Định nghĩa các biến `URL` tới trang chủ amazon, `SESSION` phục vụ ghi log

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import time, datetime
import os
# define variables 
SESSION = datetime.datetime.now().strftime("%d%m%Y%H%M")
URL = "https://www.amazon.com/"

3. Tạo file log

In [22]:
# create log
logs_file = open('logs/session{0}.log'.format(SESSION),'w+',encoding='utf-8');

4. Khởi chạy Chrome, truy cập trang chủ amazon, ghi log

In [23]:
# start webdriver Chrome
web = webdriver.Chrome(executable_path=ChromeDriverManager().install())
web.get(url=URL)
logs_file.write("------------------------------------\nStart new session: {0}\n".format(SESSION))

69

5. Danh sách các danh mục sản phẩm trên amazon

In [27]:
# get categories
_search_dropdown = Select(web.find_element(By.ID, 'searchDropdownBox'))
categories = _search_dropdown.options

# convert WebElement to String
categories = [category.text for category in categories]
# remove 'All'
categories.remove('All Departments')
logs_file.write("All category: {0}\n".format(', '.join(categories)))
logs_file.write("Total category: {0}\n".format(len(categories)))
print(categories)

['Arts & Crafts', 'Automotive', 'Baby', 'Beauty & Personal Care', 'Books', "Boys' Fashion", 'Computers', 'Deals', 'Digital Music', 'Electronics', "Girls' Fashion", 'Health & Household', 'Home & Kitchen', 'Industrial & Scientific', 'Kindle Store', 'Luggage', "Men's Fashion", 'Movies & TV', 'Music, CDs & Vinyl', 'Pet Supplies', 'Prime Video', 'Software', 'Sports & Outdoors', 'Tools & Home Improvement', 'Toys & Games', 'Video Games', "Women's Fashion"]


6. Định nghĩa các hàm để lưu đường dẫn đến các sản phẩm trên amazon

In [25]:
#get all link of products
def get_products_one_page(page, category):
    try:
        _product = web.find_elements_by_xpath("//div[contains(@class,'s-title-instructions-style')]//a")
        products_file = open('data/{0}.txt'.format(category),'a+',encoding='utf-8')
        for product in _product:
            products_file.write(str(product.get_attribute("href"))+'\n')
        products_file.close()
        logs_file.write(" ({0}:{1}) ".format(page,len(_product)))
    except:
        web.delete_all_cookies()
        web.refresh()

def get_all_products(start_page, end_page, category):
    logs_file.write("Number product on page:")
    for i in range(start_page,end_page):
        print("Save products at page {0}!".format(i))
        get_products_one_page(i,category)
        print("Save successful! Redirecting page {0}".format(int(i) + 1))
        web.find_element_by_xpath("//div[@role='navigation']//a[contains(@class,'s-pagination-next')]").click()
        time.sleep(10)

7. Lưu tất cả sản phẩm thuộc của mỗi danh mục

In [29]:
# get all products in each category
i = 'Computers'
start_page = 370 # default 1
end_page = 401 # not configure
is_start = False
# True - chuyển trang từ trang home sang trang 1 của category i
if is_start:
    # choose category and find
    _search_dropdown.select_by_visible_text(i)
    logs_file.write("--------------{0}--------------\n".format(i))
    web.find_element_by_id("nav-search-submit-button").click()
    # wait 5s to load page
    time.sleep(5)
    # click on "See all results"
    web.find_element_by_xpath('//a[span="See all results"]').click()
    # wait 5s to load page
    time.sleep(5)
else:
    get_all_products(start_page, end_page, i)

Save products at page 370!
Save successful! Redirecting page 371
Save products at page 371!
Save successful! Redirecting page 372
Save products at page 372!
Save successful! Redirecting page 373
Save products at page 373!
Save successful! Redirecting page 374
Save products at page 374!
Save successful! Redirecting page 375
Save products at page 375!
Save successful! Redirecting page 376
Save products at page 376!
Save successful! Redirecting page 377
Save products at page 377!
Save successful! Redirecting page 378
Save products at page 378!
Save successful! Redirecting page 379
Save products at page 379!
Save successful! Redirecting page 380
Save products at page 380!
Save successful! Redirecting page 381
Save products at page 381!
Save successful! Redirecting page 382
Save products at page 382!
Save successful! Redirecting page 383
Save products at page 383!
Save successful! Redirecting page 384
Save products at page 384!
Save successful! Redirecting page 385
Save products at page 385

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@role='navigation']//a[contains(@class,'s-pagination-next')]"}
  (Session info: chrome=113.0.5672.127)


8. Đóng file log và Chrome

In [21]:
logs_file.close()
web.quit()